In [37]:
bayern_raw = readcsv("Bayern Munich Player Stats.csv");
real_raw = readcsv("Real Madrid Player Stats.csv");

(m,n) = size(bayern_raw)
(m2,n2) = size(real_raw)

(26,30)

In [39]:
real_squad_table = real_raw
bayern_squad_table = bayern_raw;

In [22]:
ix = sortperm(A[1,:])

10-element Array{Int64,1}:
  7
 10
  8
  4
  6
  2
  5
  1
  9
  3

In [24]:
A[:,ix[1:3]]

2×3 Array{Float64,2}:
 0.0564732  0.0821054  0.155946
 0.262768   0.142597   0.144909

In [21]:
?sortperm

search: sortperm sortperm!



```
sortperm(v, [alg=<algorithm>,] [by=<transform>,] [lt=<comparison>,] [rev=false])
```

Return a permutation vector of indices of `v` that puts it in sorted order. Specify `alg` to choose a particular sorting algorithm (see Sorting Algorithms). `MergeSort` is used by default, and since it is stable, the resulting permutation will be the lexicographically first one that puts the input array into sorted order – i.e. indices of equal elements appear in ascending order. If you choose a non-stable sorting algorithm such as `QuickSort`, a different permutation that puts the array into order may be returned. The order is specified using the same keywords as `sort!`.

See also [`sortperm!`](:func:`sortperm!`).


In [28]:
using JuMP
rm = Model()

squad_stat_values = real_squad_table[(2:26),(4:30)]

@variable(rm, player_names[1:25])
@variable(rm, position[1:25])
@variable(rm, finalteam)

@variable(rm, gk_number)
@variable(rm, def_number)
@variable(rm, mid_number)
@variable(rm, fwd_number)
@variable(rm, count)

@constraint(rm, count<=11)

gk_number = 1
def_number = 4
mid_number = 4
fwd_number = 2
count = 1;

finalteam = Array{Any,2}(zeros(11,29))
indexes = Array{Int64}(zeros(11))

#-----------------------------------------------------------------------------------------------------------------------------
#storing player attributes

player_names = Array{String}(real_squad_table[2:26,1])
position = Array{String}(real_squad_table[2:26,3])
acceleration = Array{Int64}(real_squad_table[2:26,4])
aggression = Array{Int64}(real_squad_table[2:26,5])
agility = Array{Int64}(real_squad_table[2:26,6])
balance = Array{Int64}(real_squad_table[2:26,7])
ball_control = Array{Int64}(real_squad_table[2:26,8])
crossing = Array{Int64}(real_squad_table[2:26,9])
dribbling = Array{Int64}(real_squad_table[2:26,10])
finishing = Array{Int64}(real_squad_table[2:26,11])
gk_diving = Array{Int64}(real_squad_table[2:26,12])
gk_handling = Array{Int64}(real_squad_table[2:26,13])
gk_reflexes = Array{Int64}(real_squad_table[2:26,14])
heading_accuracy = Array{Int64}(real_squad_table[2:26,15])
interceptions = Array{Int64}(real_squad_table[2:26,16])
long_passing = Array{Int64}(real_squad_table[2:26,17])
long_shots = Array{Int64}(real_squad_table[2:26,18])
marking = Array{Int64}(real_squad_table[2:26,19])
positioning = Array{Int64}(real_squad_table[2:26,20])
reactions = Array{Int64}(real_squad_table[2:26,21])
short_passing = Array{Int64}(real_squad_table[2:26,22])
shotpower = Array{Int64}(real_squad_table[2:26,23])
sliding_tackle = Array{Int64}(real_squad_table[2:26,24])
sprint_speed = Array{Int64}(real_squad_table[2:26,25])
standing_tackle = Array{Int64}(real_squad_table[2:26,26])
stamina = Array{Int64}(real_squad_table[2:26,27])
strength = Array{Int64}(real_squad_table[2:26,28])
vision = Array{Int64}(real_squad_table[2:26,29])
volleys = Array{Int64}(real_squad_table[2:26,30])



#-----------------------------------------------------------------------------------------------------------------------------
#choosing goalkeeper
max = 0;
goalkeeper_score = 0;

for i = 1:length(player_names)
    goalkeeper_score = squad_stat_values[i,9] + squad_stat_values[i,10] + squad_stat_values[i,11]
    if goalkeeper_score>=max
        max = goalkeeper_score
        indexes[count] = i
    end
end

if count<=10
    count = count + 1;
end

#-----------------------------------------------------------------------------------------------------------------------------
#choosing defence
defender_scores = Array{Int64}(zeros(1,25))[:]

for i = 1:25
    defender_scores[i] = squad_stat_values[i,2] + squad_stat_values[i,12] + squad_stat_values[i,13] + squad_stat_values[i,16] 
    + squad_stat_values[i,21] + squad_stat_values[i,23] + squad_stat_values[i,25]
end

defender_index = sortperm(defender_scores, rev=true)
i = 1;
def_count = 0;

while def_count<=def_number-0.1
    if position[defender_index[i]] == "CB" || position[defender_index[i]] == "FB"
        indexes[count] = defender_index[i]
        def_count = def_count + 1;
        if count<=10
            count = count + 1;
        end
    end
    i=i+1
end

In [29]:
#-----------------------------------------------------------------------------------------------------------------------------
#choosing midfield
midfielder_scores = Array{Int64}(zeros(1,25))[:]

for i = 1:25
    midfielder_scores[i] = squad_stat_values[i,1] + squad_stat_values[i,3] + squad_stat_values[i,4] + squad_stat_values[i,5] + squad_stat_values[i,6] + squad_stat_values[i,7] + squad_stat_values[i,8] + squad_stat_values[i,14] + squad_stat_values[i,15] + squad_stat_values[i,17] + squad_stat_values[i,18] + squad_stat_values[i,19] + squad_stat_values[i,20] + squad_stat_values[i,26]
end

midfielder_index = sortperm(midfielder_scores, rev=true)
j = 1;
mid_count = 0;

while mid_count<=mid_number-0.1
    if position[midfielder_index[j]] == "CM" || position[midfielder_index[j]] == "WG"
        indexes[count] = midfielder_index[j]
        if count<=10
            count = count + 1;
        end
        mid_count = mid_count + 1
    end
    j=j+1
end


In [30]:
#-----------------------------------------------------------------------------------------------------------------------------
#choosing strikers
forward_scores = Array{Int64}(zeros(1,25))[:]

for i = 1:25
    forward_scores[i] = squad_stat_values[i,1] + squad_stat_values[i,3] + squad_stat_values[i,5] + squad_stat_values[i,7] + squad_stat_values[i,8] + squad_stat_values[i,12] + squad_stat_values[i,15] + squad_stat_values[i,17] + squad_stat_values[i,18] + squad_stat_values[i,20] + squad_stat_values[i,22] + squad_stat_values[i,24] + squad_stat_values[i,27]
end

forward_index = sortperm(forward_scores, rev=true)
i = 1
fwd_count = 0;

while fwd_count<=fwd_number-0.1
    if position[forward_index[i]] == "FW"
        indexes[count] = forward_index[i]
        if count<=10
            count = count + 1;
        end
        fwd_count = fwd_count+1
    end
    i=i+1
end

In [40]:
#-----------------------------------------------------------------------------------------------------------------------------
#assembling team
for i = 1:11
    finalteam[i,1] = player_names[indexes[i]]
    finalteam[i,2] = position[indexes[i]]
    finalteam[i,3] = acceleration[indexes[i]]
    finalteam[i,4] = aggression[indexes[i]]
    finalteam[i,5] = agility[indexes[i]]
    finalteam[i,6] = balance[indexes[i]]
    finalteam[i,7] = ball_control[indexes[i]]
    finalteam[i,8] = crossing[indexes[i]]
    finalteam[i,9] = dribbling[indexes[i]]
    finalteam[i,10] = finishing[indexes[i]]
    finalteam[i,11] = gk_diving[indexes[i]]
    finalteam[i,12] = gk_handling[indexes[i]]
    finalteam[i,13] = gk_reflexes[indexes[i]]
    finalteam[i,14] = heading_accuracy[indexes[i]]
    finalteam[i,15] = interceptions[indexes[i]]
    finalteam[i,16] = long_passing[indexes[i]]
    finalteam[i,17] = long_shots[indexes[i]]
    finalteam[i,18] = marking[indexes[i]]
    finalteam[i,19] = positioning[indexes[i]]
    finalteam[i,20] = reactions[indexes[i]]
    finalteam[i,21] = short_passing[indexes[i]]
    finalteam[i,22] = shotpower[indexes[i]]
    finalteam[i,23] = sliding_tackle[indexes[i]]
    finalteam[i,24] = sprint_speed[indexes[i]]
    finalteam[i,25] = standing_tackle[indexes[i]]
    finalteam[i,26] = stamina[indexes[i]]
    finalteam[i,27] = strength[indexes[i]]
    finalteam[i,28] = vision[indexes[i]]
    finalteam[i,29] = volleys[indexes[i]]
end

println("Real Madrid's team is: ")
finalteam[:,:]

Real Madrid's team is: 


11×29 Array{Any,2}:
 "Keylor Navas"       "GK"  61  32  70  …  30  21  14  52  14  41  65  54  11
 "Pepe"               "CB"  65  94  63     67  63  90  75  89  64  84  48  23
 "Sergio Ramos"       "CB"  77  84  80     76  79  90  78  89  84  81  63  66
 "Raphael Varane"     "CB"  76  73  58     72  52  85  78  86  70  84  56  42
 "Marcelo"            "FB"  77  84  77     81  83  87  80  84  82  77  75  54
 "Cristiano Ronaldo"  "WG"  91  63  90  …  82  94  23  92  31  92  80  85  88
 "Gareth Bale"        "WG"  93  65  77     86  91  52  95  55  78  80  79  76
 "Luka Modric"        "CM"  77  62  93     92  73  73  71  80  83  58  90  74
 "James Rodriguez"    "CM"  77  65  82     84  86  30  75  42  68  70  88  91
 "Karim Benzema"      "FW"  79  65  77     86  83  12  82  15  74  78  85  79
 "Alvaro Morata"      "FW"  79  54  59  …  74  78  20  86  14  68  82  76  80